# Validating Data and Predictive Pipelines
#### Making Meaningful Predictions from Data

In this notebook, I will go over basic training, testing, and validating data and move on to some pipeline practice. I will be looking at how well various words can predict a business' rating.

---

## Part 1: Training, Validating, Testing
1. **Training datasets** form the basis of a model's learning, teaching it how to estimate its output.
2. **Validation datasets** check the accuracy, quality, and integrity of a model, as well as fine-tuning any parameters that are not directly optimized.
3. **Test datasets** are usually what I am most interested in -- how does the model perform with previously unseen data that reflects real-world cases?

To learn more about the difference between these, click here: https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7

A good model is one that generalizes to new data! If a model performs well on a training set but not on new data, then it is probably **overfitting** the training data.

#### "Theorems" about these sets

* The training error increases as lambda increases
* The validation and test error are at least as large as the training error (assuming infinitely large random partitions)
* The validation/test error will usually have a “sweet spot” between under- and over-fitting

### The Data

This dataset contains 25,000 Yelp reviews of various businesses. 

### Reading the Data
First I import a few libraries.

In [1]:
import json
from collections import defaultdict
import string # Some string utilities
import random
from nltk.stem.porter import PorterStemmer # Stemming
import numpy

In [2]:
path = "./datasets/reviews.json"
f = open(path, 'rt', encoding="utf8")

# Load in JSON file
dataset = []
for i in range(25000):
    dataset.append(json.loads(f.readline()))

In [3]:
# Look at first entry in dataset
dataset[0]

{'review_id': 'QScdf32LViCASQirYGJ6hA',
 'user_id': 'ZnZmF11O42qfhhNNA5juhA',
 'business_id': 'XHYxwXtnidQsm89rE_OIUQ',
 'stars': 5.0,
 'useful': 0,
 'funny': 0,
 'cool': 0,
 'text': "WOW!! This company is amazing!!!!! As a full time working mom I can't tell you how hard it is to keep up with keeping the house clean. I am a neat freak and they came in and made my house so spotless. It is such a breath of fresh air coming home and knowing I don't have to worry about anything! My house is so shiny and it even smelled good! You guys did a fantastic job and we will definitely be contacting you again! This cleaning came at the perfect time and the cleaning ladies were so nice and friendly! Thank you so much! You have made a life long customer!",
 'date': '2016-11-22 01:48:06'}

### 1a. Examine The Dataset
#### Counting Number of Unique Words
Before I start analyzing this data, let's see how many unique words we have.

In [4]:
# Count total unique words
wordCount = defaultdict(int)
for d in dataset:
    for w in d['text'].split():
        wordCount[w] += 1

print(len(wordCount))

119350


That's a lot of words! This number of words is too many to deal with (i.e., it would result in a 119,350 dimensional feature vector if used naively). Let's try to reduce this by removing punctuation and capitalization, so that two instances of the same word will be counted as being the same even if punctuated or capitalized differently.

In [5]:
# Count total unique words, ignoring punctuation and capitalization
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in dataset:
    r = ''.join([c for c in d['text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

print(len(wordCount))

52202


Still left with a large number of words, so possibly we can reduce this number of words further if we treat different word inflections (e.g. "drinks" vs. "drinking") as being instances of the same word, by identifying their word stem (i.e., "drink"). This process is called "stemming".
I perform stemming using a stemmer from the NLTK (Natural Language Toolkit) library.

In [6]:
# Count total unique words, ignoring punctuation and capitalization and using stemming.
# This one might take a bit of time to run -- be patient!
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer() # object doing the stemming
for d in dataset:
    r = ''.join([c for c in d['text'].lower() if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w) # with stemming
        wordCount[w] += 1

print(len(wordCount))

39547


#### Extracting and Building Features from the Most Common Words
That's still a lot of words. One way to deal with this is to take the most common words and build features out of those.

First I build a few data structures to count the number of instances of each word. Here I remove punctuation and capitalization, but do not apply stemming.

In [7]:
# Count total unique words, ignoring punctuation and capitalization
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in dataset:
    r = ''.join([c for c in d['text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

Having counted the number of instances of each word, I can sort them to find the most common, and build a word index based on these frequencies. For example, the most frequent word will have index 0, the secont most frequent will have index 1, etc.

In [8]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse() # Most frequent = index 0

words = [x[1] for x in counts[:1000]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

#### Building our Bag-of-Words Feature Representation
Now that I have our dictionary of common words, we can now build a feature vector by counting how often each of these words appears in each review. This is called a **"bag-of-words" feature representation**. This results in a 1,000 dimensional feature vector (1,001 if we include an offset term).

In [9]:
# Function for building bag-of-words
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['text'].lower() if not c in punctuation])
    for w in r.split():
        if w in wordSet:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

### 1b. Build Training Dataset
I will build our training set by doing the following:
1. Shuffle entries
2. Define `X` - matrix of features
3. Define `y` - the outcomes
4. Perform least squares regression

In [10]:
random.shuffle(dataset)

In [11]:
X = [feature(d) for d in dataset] # using the bag-of-words function above

In [12]:
y = [d['stars'] for d in dataset]

In [13]:
theta,residuals,rank,s = numpy.linalg.lstsq(X, y,rcond=None)

### 1c. Visualize Important Words
Once the model has finished training, I can examine which are the most positive (or negative) words by looking at the largest (or smallest) corresponding values for theta To do so, let's sort the words based on their corresponding weights from `theta`:

In [14]:
wordWeights = list(zip(theta, words + ['offset']))
wordWeights.sort()

In [15]:
# 10 most negative words
wordWeights[:10]

[(-0.7553245155694435, 'worst'),
 (-0.7389947690061027, 'horrible'),
 (-0.6005449383926107, 'terrible'),
 (-0.5290251025492264, 'rude'),
 (-0.5215853670104232, 'poor'),
 (-0.48335065677885247, 'bland'),
 (-0.48050793069310804, 'awful'),
 (-0.4623144737888688, 'overpriced'),
 (-0.46124438293703945, 'disappointing'),
 (-0.320489881944537, 'dirty')]

In [16]:
# 9 most positive words (10th is offset term)
wordWeights[-10:]

[(0.2507418090182752, 'best'),
 (0.2710529489617402, 'reasonable'),
 (0.27383412028756937, 'above'),
 (0.27552216151668696, 'professional'),
 (0.3063027581586295, 'awesome'),
 (0.3110107093777941, 'amazing'),
 (0.3135436872881369, 'excellent'),
 (0.3153643923320769, 'pleased'),
 (0.3190050881565601, 'outstanding'),
 (3.6422465896845466, 'offset')]

### 1d. Include Regularizer
**Regularization** is the process of penalizing model complexity during training. In this case, though the above model was effective, it was also very high dimensional, and thus may have been prone to **overfitting**. We can try to address this by adding a regularizer to our model.

The "Ridge" class from `sklearn` implements a least squares regression model (as in our example above) that includes a regularizer. The strength of the regularizer is controlled by the parameter `alpha` (equivalent to `lambda` in the course lectures). Otherwise, fitting the ridge regression model is exactly the same as fitting a regular least squares model! 

Here is the method call I am using: `model = linear_model.Ridge(1.0, fit_intercept=False)`

**Note the two extra parameters:** The first is the regularization strength (`alpha`). The second indicates I do not want the model to fit an intercept (since the feature vector already includes one).

In [17]:
from sklearn import linear_model

In [18]:
help(linear_model.Ridge)

Help on class Ridge in module sklearn.linear_model._ridge:

class Ridge(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidge)
 |  Ridge(alpha=1.0, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, solver='auto', random_state=None)
 |  
 |  Linear least squares with l2 regularization.
 |  
 |  Minimizes the objective function::
 |  
 |  ||y - Xw||^2_2 + alpha * ||w||^2_2
 |  
 |  This model solves a regression model where the loss function is
 |  the linear least squares function and regularization is given by
 |  the l2-norm. Also known as Ridge Regression or Tikhonov regularization.
 |  This estimator has built-in support for multi-variate regression
 |  (i.e., when y is a 2d-array of shape (n_samples, n_targets)).
 |  
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |  
 |  Parameters
 |  ----------
 |  alpha : {float, array-like of shape (n_targets,)}, default=1.0
 |      Regularization strength; must be a positive float. Regul

In [19]:
model = linear_model.Ridge(1.0, fit_intercept=False)
model.fit(X, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

Let's examine the coefficients learned by our new regularized model.

In [20]:
theta = model.coef_

In [21]:
wordWeights = list(zip(theta, words + ['offset']))
wordWeights.sort()

In [22]:
# 10 most negative words
wordWeights[:10]

[(-0.7542478890211676, 'worst'),
 (-0.7380238948354739, 'horrible'),
 (-0.5997972082231678, 'terrible'),
 (-0.528644144203692, 'rude'),
 (-0.5206526415791057, 'poor'),
 (-0.48254221668243685, 'bland'),
 (-0.4793726507312775, 'awful'),
 (-0.4608536828058321, 'overpriced'),
 (-0.459806274640262, 'disappointing'),
 (-0.3197940121855978, 'dirty')]

In [23]:
# 9 most positive words (10th is offset term)
wordWeights[-10:]

[(0.25089466451190345, 'best'),
 (0.2707656635951766, 'reasonable'),
 (0.27319251332630284, 'above'),
 (0.27536923153242904, 'professional'),
 (0.30631213715059025, 'awesome'),
 (0.31111856323805875, 'amazing'),
 (0.3135411208752763, 'excellent'),
 (0.3142912016174824, 'pleased'),
 (0.31821406027182847, 'outstanding'),
 (3.6416051508951472, 'offset')]

---
## Part 2: Regression Diagnostics: MSE and R^2 Review
Let's step away from programming for a moment. How do we evaluate our regressors? 

### 2a. MSE (Mean Squared Error)
What is the MSE (which we have already been using) and its relationship to the R^2 statistic? I start by extracting the predictions from the model and computing their squared differences:

In [24]:
predictions = model.predict(X)

In [25]:
differences = [(x-y)**2 for (x,y) in zip(predictions,y)]

The **MSE** is just the average (mean) of these squared differences, which we can compute with a few trivial Python commands:

In [26]:
MSE = sum(differences) / len(differences)
print("MSE = " + str(MSE))

MSE = 0.9237226689725614


### 2b. R^2
the **R^2** (and the **FVU**, or "Fraction of Variance Unexplained") normalize the **Mean Squared Error** based on the variance of the data:

In [27]:
FVU = MSE / numpy.var(y)
R2 = 1 - FVU
print("R2 = " + str(R2))

R2 = 0.5582919710243408


---

## Part 3: Predictive Pipelines

In its most basic form, a pipeline links steps together. In machine learning and data science, pipelines allow you transform data from one representation to another through a series of steps. 

A lower-level example can be found in Unix, where you can *pipeline* the *output* of some command as an *input* some other command with the `|` (e.g. `cat words.txt | wc -c` will result in the number of characters in `words.txt`). In this notebook, I will implement a regularization pipeline to help us train, test, and validate data, where (as stated before) **regularization** is the process of penalizing model complexity during training.

Let's import libraries and read our dataset in again.

In [28]:
import json
from collections import defaultdict
import string
import random

In [29]:
path = "./datasets/reviews.json"
f = open(path, 'rt', encoding="utf8")

# Load in JSON file
dataset = []
for i in range(25000):
    dataset.append(json.loads(f.readline()))

### 3a. Rebuild Common Word Features
This is identical to what we did in Part 1.

In [30]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)

for d in dataset:
  r = ''.join([c for c in d['text'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse() # Highest frequency = index 0

words = [x[1] for x in counts[:1000]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [31]:
# Bag-of-words function
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['text'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

### 3b. Build the Validation Set
Again I split our data, but this time I split it into _three_ parts - a training, a validation, and a test component. Recall that validation sets check the accuracy, quality, and integrity of a model, as well as fine-tuning any parameters that are not directly optimized.

Let's shuffle the dataset and define `X` and `y` again.

In [32]:
random.shuffle(dataset)
X = [feature(d) for d in dataset]
y = [d['stars'] for d in dataset]

In this notebook I'll use half of our data (and labels) for **training**, the next quarter for **validation**, and the final quarter for **testing**.

In [33]:
N = len(X)

X_train = X[:N//2]
X_valid = X[N//2:3*N//4]
X_test = X[3*N//4:]

y_train = y[:N//2]
y_valid = y[N//2:3*N//4]
y_test = y[3*N//4:]

In [34]:
# Examine size of data
len(X), len(X_train), len(X_valid), len(X_test)

(25000, 12500, 6250, 6250)

### 3c. Train the Model
Again I'll train a model based on regularized (Ridge) regression. My MSE function is the same as before, but this time for convience takes an (already trained) model as an input parameter.

#### Finally, to implement the pipeline, I:
1. Iterate through various values of lambda
2. Fit a ridge regression model for each of these values
3. Evaluate the performance of this model on the validation set
4. Keep track of which model is the best we've seen so far (on the validation set)

In [35]:
from sklearn import linear_model

In [36]:
# Function to find Mean Squared Error
def MSE(model, X, y):
    predictions = model.predict(X)
    differences = [(a-b)**2 for (a,b) in zip(predictions, y)]
    return sum(differences) / len(differences)

In [37]:
# Variables to keep track of the current best model and MSE
bestModel = None 
bestMSE = None

In [38]:
# Train and fine-tune model with training and validation sets
for lamb in [0.01, 0.1, 1, 10, 100]:
    model = linear_model.Ridge(lamb, fit_intercept=False)
    model.fit(X_train, y_train)
    
    mseTrain = MSE(model, X_train, y_train)
    mseValid = MSE(model, X_valid, y_valid)
    
    print("lambda = " + str(lamb) + ", training/validation error = " +
          str(mseTrain) + '/' + str(mseValid))
    if not bestModel or mseValid < bestMSE:
        bestModel = model
        bestMSE = mseValid

lambda = 0.01, training/validation error = 0.8750598201114304/1.0432668230964726
lambda = 0.1, training/validation error = 0.8750598593116713/1.0431903936265863
lambda = 1, training/validation error = 0.8750637401231084/1.0424352096193081
lambda = 10, training/validation error = 0.8754164032180508/1.0357125897260049
lambda = 100, training/validation error = 0.8935572875821944/1.0107344692176796


Finally, I evaluate the performance of my best model on the **test** set. Note that this is the only time throughout the entire pipeline that I examine the test data!

In [39]:
mseTest = MSE(bestModel, X_test, y_test)
print("test error = " + str(mseTest))

test error = 1.0723185592283093
